In [ ]:
%pip install torch torchaudio onnxruntime silero-vad

In [2]:
from silero_vad import load_silero_vad, read_audio, get_speech_timestamps

model = load_silero_vad()
wav = read_audio('./test.mp3')
speech_timestamps = get_speech_timestamps(
  wav,
  model,
  return_seconds=True,
)

print(speech_timestamps)

[{'start': 0.2, 'end': 7.5}, {'start': 7.6, 'end': 16.9}, {'start': 17.0, 'end': 20.4}, {'start': 20.6, 'end': 31.3}, {'start': 31.6, 'end': 35.9}, {'start': 36.0, 'end': 38.2}, {'start': 38.3, 'end': 41.7}, {'start': 41.8, 'end': 43.2}, {'start': 43.3, 'end': 48.4}, {'start': 48.6, 'end': 54.5}, {'start': 54.6, 'end': 62.8}, {'start': 62.9, 'end': 73.2}, {'start': 73.3, 'end': 74.8}, {'start': 75.0, 'end': 85.0}, {'start': 85.3, 'end': 88.1}, {'start': 88.2, 'end': 92.8}, {'start': 92.9, 'end': 95.9}, {'start': 96.0, 'end': 98.9}, {'start': 99.0, 'end': 110.6}, {'start': 110.7, 'end': 115.9}, {'start': 116.0, 'end': 121.2}, {'start': 121.5, 'end': 136.3}, {'start': 136.6, 'end': 140.4}, {'start': 140.7, 'end': 143.2}, {'start': 143.3, 'end': 144.9}, {'start': 145.1, 'end': 153.8}, {'start': 153.9, 'end': 156.4}, {'start': 156.7, 'end': 160.5}, {'start': 160.8, 'end': 165.2}, {'start': 165.3, 'end': 167.9}, {'start': 168.3, 'end': 172.5}, {'start': 172.7, 'end': 184.2}, {'start': 184.3

In [15]:
for segment in speech_timestamps:
    m_start = segment['start']//60
    s_start = segment['start']%60
    m_end = segment['end']//60
    s_end = segment['end']%60
    print(f"\nstart:{m_start}m {s_start}s\nend:{m_end}m {s_end}s\n")


start:0.0m 0.2s
end:0.0m 7.5s


start:0.0m 7.6s
end:0.0m 16.9s


start:0.0m 17.0s
end:0.0m 20.4s


start:0.0m 20.6s
end:0.0m 31.3s


start:0.0m 31.6s
end:0.0m 35.9s


start:0.0m 36.0s
end:0.0m 38.2s


start:0.0m 38.3s
end:0.0m 41.7s


start:0.0m 41.8s
end:0.0m 43.2s


start:0.0m 43.3s
end:0.0m 48.4s


start:0.0m 48.6s
end:0.0m 54.5s


start:0.0m 54.6s
end:1.0m 2.799999999999997s


start:1.0m 2.8999999999999986s
end:1.0m 13.200000000000003s


start:1.0m 13.299999999999997s
end:1.0m 14.799999999999997s


start:1.0m 15.0s
end:1.0m 25.0s


start:1.0m 25.299999999999997s
end:1.0m 28.099999999999994s


start:1.0m 28.200000000000003s
end:1.0m 32.8s


start:1.0m 32.900000000000006s
end:1.0m 35.900000000000006s


start:1.0m 36.0s
end:1.0m 38.900000000000006s


start:1.0m 39.0s
end:1.0m 50.599999999999994s


start:1.0m 50.7s
end:1.0m 55.900000000000006s


start:1.0m 56.0s
end:2.0m 1.2000000000000028s


start:2.0m 1.5s
end:2.0m 16.30000000000001s


start:2.0m 16.599999999999994s
end:2.0m 20.4000

In [16]:
from torchaudio.io import play_audio
import sounddevice as sd

sd.play(wav, 48000)
sd.wait()

<sounddevice.CallbackFlags: output underflow>

In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
torch_dtype, device

(torch.float32, 'cpu')

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

In [ ]:
processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True,
)

In [ ]:
dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

In [ ]:
result = pipe(sample)
print(result.keys())
print(result["text"])